# Clase: Toma de decisiones con IA

Ejemplo simple de **series de tiempo** para mostrar en clases.

Escenario ficticio: *número de tickets diarios en una mesa de ayuda ciudadana*.

## 1. Importar librerías

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error


## 2. Crear una serie de tiempo ficticia

Simularemos 730 días (2 años) de tickets diarios con:
- Tendencia suave al alza.
- Comportamiento semanal (más tickets de lunes a viernes, menos el fin de semana).
- Algo de ruido aleatorio.

In [ ]:
np.random.seed(42)

# 2 años de datos diarios
fechas = pd.date_range(start="2023-01-01", periods=730, freq="D")

# Tendencia lineal
tendencia = np.linspace(30, 80, 730)

# Componente semanal: más carga de lunes (0) a viernes (4), menos fin de semana
dia_semana = fechas.dayofweek
componente_semanal = np.where(dia_semana < 5, 15, -10)

# Ruido aleatorio
ruido = np.random.normal(0, 5, size=730)

tickets = tendencia + componente_semanal + ruido
tickets = np.clip(tickets, 5, None)  # no menos de 5 tickets

df_ts = pd.DataFrame({
    "fecha": fechas,
    "tickets": tickets,
    "dia_semana": dia_semana
})

df_ts.head()


## 3. Crear variables para el modelo

Usaremos como insumos:
- `tickets_ayer`: número de tickets del día anterior (retardo de 1 día).
- `dia_semana`: día de la semana (0=lunes,...,6=domingo).

In [ ]:
# Crear variable de retardo (lag)
df_ts["tickets_ayer"] = df_ts["tickets"].shift(1)
df_ts = df_ts.dropna().reset_index(drop=True)

df_ts.head()


## 4. Separar entrenamiento y prueba

Usaremos los primeros ~80% de días para entrenar y el resto para probar (respetando el orden temporal).

In [ ]:
n = len(df_ts)
n_train = int(n * 0.8)

train = df_ts.iloc[:n_train]
test = df_ts.iloc[n_train:]

X_train = train[["tickets_ayer", "dia_semana"]]
y_train = train["tickets"]

X_test = test[["tickets_ayer", "dia_semana"]]
y_test = test["tickets"]

n, n_train, len(test)


## 5. Entrenar un modelo sencillo de regresión

El modelo aprenderá a predecir `tickets` en función de `tickets_ayer` y `dia_semana`.

In [ ]:
modelo_ts = LinearRegression()
modelo_ts.fit(X_train, y_train)

print("Modelo de series de tiempo entrenado 👍")


## 6. Evaluar el modelo

In [ ]:
y_pred = modelo_ts.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
print(f"Error absoluto medio (MAE): {mae:.2f} tickets")        


## 7. Comparar valores reales vs. predichos en los últimos días

In [ ]:
comparacion = test.copy()
comparacion["prediccion"] = y_pred

comparacion.tail(10)


## 8. Visualizar la serie (opcional para la clase)

Podemos ver en un gráfico los valores reales y las predicciones en el tramo de prueba.

In [ ]:
plt.figure(figsize=(10,4))
plt.plot(test["fecha"], y_test, label="Real")
plt.plot(test["fecha"], y_pred, label="Predicción")
plt.xlabel("Fecha")
plt.ylabel("Tickets diarios")
plt.title("Serie de tiempo: tickets reales vs. predichos (tramo de prueba)")
plt.legend()
plt.tight_layout()
plt.show()


## 9. Resumen para la clase

- Esta es una **serie de tiempo**: datos ordenados día a día.
- El modelo usa el valor de ayer y el día de la semana para predecir hoy.
- Sirve para **anticipar carga de trabajo**, planificar turnos y dimensionar recursos en una mesa de ayuda u oficina de atención ciudadana.

La misma lógica se usa para demanda de transporte, consultas en salud, consumo energético, etc.